# **Colab Data Importing ZIP files and Cleaning with Python - Project2025-N10a**

**Created by** Cristina

**Created on** 10/15/2025

**Description:** Importing a zipfile straight from an s3 bucket on AWS, extracting, taking to dataframes and transforming data to be then used in PowerBI

Also zipfile github to dataframes

**Data Source:** zipfile in an s3 bucket on AWS; also zipfile in github

**Challenges:** setting the path to import the zipfile can be tricky.

## Loading ZIP file from AWS and Inspecting the Data




In [27]:
import requests
import zipfile
import io
import pandas as pd
from google.colab import files # for downloading from colab to local machine

# ZIP file from AWS to dictionary of df (one df per csv)

# ✅ Use the direct ZIP URL (not the YouTube redirect)
# url = "https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbVZ6WkdJY2V5WXhsaTFxOEd3amVrZ0dUenFrd3xBQ3Jtc0trX054cTZlSURBeXFUblZYV3BJZ216bmFvM2ZGNWZqS1BSeDdtc1hDUDJsczB4X3F3X0tWVlAwSHdacUcxTUpaQkgtbVIyb1N2alJCQzA0eEg0YUdrMERnVlZQdWFoYkRHS0o3WmRqT1R0dHZHSF92OA&q=https%3A%2F%2Fmaven-datasets.s3.amazonaws.com%2FAirline%2BFlight%2BDelays%2FAirline%2BFlight%2BDelays.zip&v=aLV4Qe60VK4"

#clean the path:
url = "https://maven-datasets.s3.amazonaws.com/Airline+Flight+Delays/Airline+Flight+Delays.zip"

headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    if response.content[:4] == b'PK\x03\x04':  # check if it's a valid zip (magic number)
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            print("ZIP file contents:", zip_file.namelist())

            dataframes = {}
            for name in zip_file.namelist():
                if name.endswith('.csv'):
                    with zip_file.open(name) as f:
                        df = pd.read_csv(f)
                        dataframes[name] = df
                        print(f"\nPreview of {name}:")
                        print(df.head())
            # flights is a huge table, so data reduction
            flights_df = dataframes['flights.csv']
            flights_df.info()
            flights_df=flights_df[['YEAR', 'MONTH', 'DAY_OF_WEEK', 'AIRLINE','ORIGIN_AIRPORT', 'DEPARTURE_DELAY', 'CANCELLED', 'CANCELLATION_REASON']]
            flights_df.dropna(inplace=True)
            flights_df.duplicated().sum()
            flights_df.drop_duplicates(inplace=True)

            print (flights_df.head())
            flights_df.to_csv('flights_clean.csv', index=False)
            download_file = files.download('flights_clean.csv')
    else:
        print("Downloaded file is not a valid ZIP.")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


ZIP file contents: ['airlines.csv', 'airports.csv', 'cancellation_codes.csv', 'Flight Status Dashboard.pbix', 'flights.csv']

Preview of airlines.csv:
  IATA_CODE                 AIRLINE
0        UA   United Air Lines Inc.
1        AA  American Airlines Inc.
2        US         US Airways Inc.
3        F9  Frontier Airlines Inc.
4        B6         JetBlue Airways

Preview of airports.csv:
  IATA_CODE                                           AIRPORT  \
0       ATL  Hartsfield-Jackson Atlanta International Airport   
1       DEN                      Denver International Airport   
2       DFW           Dallas/Fort Worth International Airport   
3       IAH              George Bush Intercontinental Airport   
4       LAS                    McCarran International Airport   

                CITY STATE COUNTRY  LATITUDE  LONGITUDE  
0            Atlanta    GA     USA  33.64044  -84.42694  
1             Denver    CO     USA  39.85841 -104.66700  
2  Dallas-Fort Worth    TX     USA  32.895

/tmp/ipython-input-3328138402.py:27: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)



Preview of flights.csv:
   Unnamed: 0  YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER  \
0           1  2015      1    1            4      AA           2336   
1           2  2015      1    1            4      US            840   
2           3  2015      1    1            4      AA            258   
3           5  2015      1    1            4      DL            806   
4           6  2015      1    1            4      NK            612   

  TAIL_NUMBER ORIGIN_AIRPORT DESTINATION_AIRPORT  ...  ARRIVAL_TIME  \
0      N3KUAA            LAX                 PBI  ...         741.0   
1      N171US            SFO                 CLT  ...         811.0   
2      N3HYAA            LAX                 MIA  ...         756.0   
3      N3730B            SFO                 MSP  ...         610.0   
4      N635NK            LAS                 MSP  ...         509.0   

   ARRIVAL_DELAY  DIVERTED  CANCELLED  CANCELLATION_REASON  AIR_SYSTEM_DELAY  \
0           -9.0         0          0    

/tmp/ipython-input-3328138402.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_df.dropna(inplace=True)
/tmp/ipython-input-3328138402.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_df.drop_duplicates(inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Loading ZIP file from GITHUB

In [28]:
import requests
import zipfile
import io

# Gets ZIP file from github, and load the CSV files into a dictionary of dataframes (name of CSV and associated df)

# Step 1: Download the zipfile from a url
# this url does not work, as it wants the link to the raw file
#url = 'https://github.com/cristina0000/2025_git_projects/blob/main/p10_PowerBI/airlines%20-%20Copy.zip'

# clean the path: for github zipfile link change blob -> raw in the link and it now works:
url='https://github.com/cristina0000/2025_git_projects/raw/main/p10_PowerBI/airlines%20-%20Copy.zip'
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

# Check for a successful response
if response.status_code == 200:
    content_type = response.headers.get('Content-Type', '')
    if 'zip' in content_type or response.content[:4] == b'PK\x03\x04':  # ZIP magic number
        try:
            # Step 2: Load the ZIP file into memory
            with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
                # List the contents
                print("ZIP file opened successfully.")
                print("Files in the ZIP:")
                print(zip_file.namelist())

                # Step 3: Extract and read csv files:
                csv_files = [name for name in zip_file.namelist() if name.endswith('.csv')]
                # Create a dictionary of key value items, where key is csv, and value is the associated df
                dataframes = {}

                for csv_file in csv_files:
                    with zip_file.open(csv_file) as f:
                        df = pd.read_csv(f)
                        dataframes[csv_file] = df

                # Print the first rows of each CSV:
                for name, df in dataframes.items():
                    print(f"\n{name} preview:")
                    print(df.head())
                # Optional other things to do:
                # Make any transformations necessary, such as removing columns, data type change, removing duplicates, removing nulls
                df_clean = dataframes['airports - Copy.csv'].drop (['LONGITUDE'], axis = 1)
                print (df_clean.head())
                # Load into a database

                # Download the CSV files to local computer

        except zipfile.BadZipFile:
            print("Error: File content is not a valid ZIP.")
    else:
        print("Downloaded file is not a ZIP. Content-Type:", content_type)
        print(response.content[:200])  # Inspect what's actually there
else:
    print(f"Failed to download file. Status code: {response.status_code}")

ZIP file opened successfully.
Files in the ZIP:
['airlines - Copy.csv', 'airports - Copy.csv', 'cancellation_codes - Copy.csv']

airlines - Copy.csv preview:
  IATA_CODE                 AIRLINE
0        UA   United Air Lines Inc.
1        AA  American Airlines Inc.
2        US         US Airways Inc.
3        F9  Frontier Airlines Inc.
4        B6         JetBlue Airways

airports - Copy.csv preview:
  IATA_CODE                                           AIRPORT  \
0       ATL  Hartsfield-Jackson Atlanta International Airport   
1       DEN                      Denver International Airport   
2       DFW           Dallas/Fort Worth International Airport   
3       IAH              George Bush Intercontinental Airport   
4       LAS                    McCarran International Airport   

                CITY STATE COUNTRY  LATITUDE  LONGITUDE  
0            Atlanta    GA     USA  33.64044  -84.42694  
1             Denver    CO     USA  39.85841 -104.66700  
2  Dallas-Fort Worth    TX     

In [29]:
import requests
import zipfile
import io
import pandas as pd

# This is a simpler version without the bad ZIP check:

# Step 1: Download the ZIP file from a URL
url = 'https://github.com/cristina0000/2025_git_projects/raw/main/p10_PowerBI/airlines%20-%20Copy.zip'  # replace with your actual URL
response = requests.get(url)
print(response.headers.get('Content-Type'))

# Check for a successful response
if response.status_code == 200:
    print("status code is 200")
    # Step 2: Load the ZIP file into memory
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
        # List the contents
        print("Files in the ZIP:")
        print(zip_file.namelist())

        # Step 3: Extract and read CSV files
        csv_files = [name for name in zip_file.namelist() if name.endswith('.csv')]
        dataframes = {}

        for csv_file in csv_files:
            with zip_file.open(csv_file) as f:
                df = pd.read_csv(f)
                dataframes[csv_file] = df

        # Example: print the first few rows of each CSV
        for name, df in dataframes.items():
            print(f"\n{name} preview:")
            print(df.head())
else:
    print(f"Failed to download file: {response.status_code}")


application/zip
status code is 200
Files in the ZIP:
['airlines - Copy.csv', 'airports - Copy.csv', 'cancellation_codes - Copy.csv']

airlines - Copy.csv preview:
  IATA_CODE                 AIRLINE
0        UA   United Air Lines Inc.
1        AA  American Airlines Inc.
2        US         US Airways Inc.
3        F9  Frontier Airlines Inc.
4        B6         JetBlue Airways

airports - Copy.csv preview:
  IATA_CODE                                           AIRPORT  \
0       ATL  Hartsfield-Jackson Atlanta International Airport   
1       DEN                      Denver International Airport   
2       DFW           Dallas/Fort Worth International Airport   
3       IAH              George Bush Intercontinental Airport   
4       LAS                    McCarran International Airport   

                CITY STATE COUNTRY  LATITUDE  LONGITUDE  
0            Atlanta    GA     USA  33.64044  -84.42694  
1             Denver    CO     USA  39.85841 -104.66700  
2  Dallas-Fort Worth    TX